**Code to download hourly electricity demand data from the Albanian TSO (OST).**

In [1]:
import os
import requests
import pandas as pd
from io import BytesIO
from openpyxl import load_workbook
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


# Constants
START_YEAR = 2025  # Only for 2025
END_YEAR = 2025    # Only for 2025
OUTPUT_DIR = "Downloaded_Data"
BASE_URL = "https://ost.al/wp-content/uploads"
MAX_WORKERS = 16  # Number of threads

# Create output folder if not exists
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Function to process one date
def process_date(args):
    year, month, day = args
    results = []

    for upload_month in [month, month + 1]:
        if upload_month > 12:
            continue

        date_str = f"{day:02d}.{month:02d}.{year}"
        file_url = f"{BASE_URL}/{year}/{upload_month:02d}/Publikimi-te-dhenave-{date_str}.xlsx"
        file_url_variation = f"{BASE_URL}/{year}/{upload_month:02d}/Publikimi-te-dhenave-{date_str}-*.xlsx"  # Variation for files with suffixes

        try:
            # Try the exact file first
            response = requests.get(file_url, timeout=10)
            if response.status_code == 200:
                # Process the file
                wb = load_workbook(BytesIO(response.content), data_only=True)
                sheet = wb['Publikime AL']
                values = [sheet[f"F{i}"].value for i in range(160, 184)]

                for hour, demand in enumerate(values, start=1):
                    results.append({
                        "date": f"{year}-{month:02d}-{day:02d}",
                        "hour": hour,
                        "demand": demand
                    })
                return results
        except Exception:
            # If the exact match fails, try the variation with any suffix
            try:
                response = requests.get(file_url_variation, timeout=10)
                if response.status_code == 200:
                    # Process the file
                    wb = load_workbook(BytesIO(response.content), data_only=True)
                    sheet = wb['Publikime AL']
                    values = [sheet[f"F{i}"].value for i in range(160, 184)]

                    for hour, demand in enumerate(values, start=1):
                        results.append({
                            "date": f"{year}-{month:02d}-{day:02d}",
                            "hour": hour,
                            "demand": demand
                        })
                    return results
            except Exception as e:
                continue  # If no file is found, move to the next date

    return []  # If nothing worked


# Generate valid date combinations
date_combos = []
for year in range(START_YEAR, END_YEAR + 1):
    for month in range(1, 13):
        for day in range(1, 32):
            try:
                datetime(year, month, day)
                date_combos.append((year, month, day))
            except ValueError:
                continue

# Main execution
if __name__ == "__main__":
    print("Starting threaded downloads and processing...")

    all_results = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_date = {executor.submit(process_date, args): args for args in date_combos}
        for future in tqdm(as_completed(future_to_date), total=len(future_to_date)):
            result = future.result()
            if result:
                all_results.extend(result)

    # Create a complete date-hour grid for all years (now only 24 hours per day)
    full_index = []

    for year in range(START_YEAR, END_YEAR + 1):
        start = datetime(year, 1, 1)
        end = datetime(year + 1, 1, 1)
        date_range = pd.date_range(start, end, freq='H')  # Removed closed='left'

        for ts in date_range:
            if ts.hour < 24:  # Ensure only 24 hours are used
                full_index.append({
                    "date": ts.strftime("%Y-%m-%d"),
                    "hour": ts.hour + 1  # Shift hours from 0-23 to 1-24
                })

    full_df = pd.DataFrame(full_index)

    # Convert collected data into DataFrame
    df = pd.DataFrame(all_results)

    # Merge the full grid with the actual data
    merged_df = pd.merge(full_df, df, on=["date", "hour"], how="left")

    # Optionally, fill missing demand with NaN or 0
    merged_df['demand'] = merged_df['demand'].fillna(0)  # Use 0 or NaN depending on your preference

    # Save to CSV
    merged_df.to_csv(os.path.join(OUTPUT_DIR, "long_format_demand_data_filled.csv"), index=False)
    print("✅ Final output with all dates filled saved as long_format_demand_data_filled.csv")

Starting threaded downloads and processing...


100%|██████████| 365/365 [01:46<00:00,  3.41it/s]

✅ Final output with all dates filled saved as long_format_demand_data_filled.csv



/var/folders/p8/lwc0sjsd4g3brnjz6n_q8chc0000gn/T/ipykernel_24074/1431091295.py:106: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start, end, freq='H')  # Removed closed='left'
